## Computational Theory Assessment

In [136]:
import numpy as np

## Problem 1: Binary Words and Operations

##### **Brief:** Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard (see page 10). Document each function with a clear docstring, explain its purpose and behaviour in Markdown, and test it with appropriate examples to verify correctness.

## Problem 1 Introduction:

**Based on the Secure Hash Standard, FIPS PUB 180-4**

The Secure Hash Standard (FIPS 180-4) defines a family of cryptographic hash algorithms, including **SHA-224, SHA-256, SHA-384,** and **SHA-512**. These algorithms use carefully designed bitwise operations to achieve diffusion and non-linearity, which are critical for cryptographic security.
**SHA-256** in particular relies on a fixed set of 32-bit operations such as:
* Bitwise logical functions (Ch, Maj, Parity)
* Fixed right rotations
* Logical right shifts
* Large and small Σ (“Sigma”) functions
* 32-bit unsigned integer arithmetic

These operations together form the backbone of the **SHA-256** compression function and message schedule expansion.
The functions below are helper utilities that directly implement the primitives defined in **Section 4 of the Secure Hash Standard**.


#### **unsigned_32(x)**
Casts a value to a NumPy uint32. Ensures all arithmetic and bitwise operations stay within 32-bit unsigned integer bounds (as required by SHA-256)

SHA-256 uses modulo 2^32 arithmetic for every operation, so consistently forcing values into a 32-bit type ensures:
* Correct overflow behaviour
* Proper masking of intermediate values
* Reproducibility across platforms
This step is essential because Python’s integers are unbounded by default.

In [137]:
def unsigned_32(x):
    return np.uint32(x);

#### **Parity(x, y, z)**
Implements the parity **XOR** function: **x ⊕ y ⊕ z**. 

Parity is a simple nonlinear combination used in some hashing and cryptographic constructions (although SHA-256 does not use Parity, it appears in SHA-1).It serves as a symmetric bitwise mixing function —> each output bit is 1 if an odd number of inputs have that bit set.

In [138]:
def Parity(x, y, z):
    return unsigned_32(x) ^ unsigned_32(y) ^ unsigned_32(z)

#### **Ch(x, y, z)** — Choose Function
SHA-256 "choice" function: **Ch(x,y,z)=(x∧y)⊕(¬x∧z)** -> as defined in FIPS 180-4, Section 4.1.2. 

Selects bits from y when x bit is 1, otherwise from z.Models a bitwise conditional.
This function acts like a **bitwise selector**:
* If a bit in x is 1 → choose the corresponding bit from y
* If the bit in x is 0 → choose the bit from z

**Cryptographic purpose:**
* Adds nonlinearity
* Makes the output depend on specific input bits
* Prevents simple algebraic prediction of internal states

Ch is used heavily in the main SHA-256 compression loop.

In [139]:
def Ch(x, y, z):
    return unsigned_32((unsigned_32(x) & unsigned_32(y)) ^ (~unsigned_32(x) & unsigned_32(z)))

#### **Maj(x, y, z)** — Majority Function
SHA-256 "majority" function: **Maj(x,y,z)=(x∧y)⊕(x∧z)⊕(y∧z)**

Each output bit becomes whichever value (0 or 1) appears at least twice among the input bits -> This makes it a majority vote.

**Cryptographic purpose:**
* Mixes three internal state values
* Propagates structure from multiple inputs
* Resists bit-flipping predictability

**Maj** is also used in every round of the compression function.

In [140]:
def Maj(x, y, z):
    return unsigned_32((unsigned_32(x) & unsigned_32(y)) ^ (unsigned_32(x) & unsigned_32(z)) ^ (unsigned_32(y) & unsigned_32(z)))

#### **rotr(x, n)** — Right Rotation
Performs a 32-bit right rotate: **(x≫n)∣(x≪(32−n))**

Rotates bits instead of shifting in zeros. Core operation in SHA-256’s mixing steps. Different from a logical shift, rotation wraps around the dropped bits.

**Why rotations matter:**
* They are nonlinear with respect to shifts
* They ensure every output bit depends on multiple positions of the input
* They preserve all bits (unlike shifts, which lose information)

Rotations are a core building block of the **Σ** and **σ** functions.

In [141]:
def rotr(x, n):
    x = unsigned_32(x)
    return unsigned_32((x >> n) | (x << unsigned_32(32 - n)))

#### **Sigma0(x)** — Uppercase **Σ₀**
Large rotation-based mixing function: **(x)=ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)**

**Purpose:** This is a large rotation mix function that produces heavy diffusion across the internal state. It is applied repeatedly inside the compression function.

In [142]:
def Sigma0(x):
    return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)

#### **Sigma1(x)** — Uppercase **Σ₁**
Large rotation-based mixing function: **(x)=ROTR^6(x) ⊕ ROTR^11(x) ⊕ ROTR^25(x)**

**Purpose:** Just like Σ₀, this function spreads bits widely and mixes state words to avoid linear relationships.

In [143]:
def Sigma1(x):
    return unsigned_32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

#### **sigma0(x)** — Lowercase **σ₀**
Small mixing function using rotations and shifts: **(x)=ROTR^7(x) ⊕ ROTR^18(x) ⊕ (x≫3)**

Used when expanding the message schedule array.

**Purpose:** Mix input words to create additional message schedule values & introduce nonlinearity early, before compression

**Note:** uses a logical right shift in addition to rotations.

In [144]:
def sigma0(x):
    x = unsigned_32(x)
    return unsigned_32(rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

#### sigma1(x) — Lowercase σ₁
Small mixing function using rotations and shifts: **(x)=ROTR^17(x) ⊕ ROTR^19(x) ⊕ (x≫10)**

**Purpose:** Also part of the message schedule expansion. Helps propagate entropy from earlier message words into later ones.

In [145]:
def sigma1(x):
    x = unsigned_32(x)
    return unsigned_32(rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10))

## Problem 2: Fractional Parts of Cube Roots

In [146]:
def primes(n):
    primes = []
    num = 2
    while len(primes) < n:
        for i in range(2,num):
            if num % i == 0:
                break
        else:
            primes.append(num)
        num += 1
    return primes

def cube_root(primes):
    cube_roots = [] 
    cube_roots = np.cbrt(primes)
    return cube_roots

def frac_32():
    p = primes(64)
    cube_roots = cube_root(p)
    frac, _= np.modf(cube_roots)
    constants = np.floor(frac * (2**32)).astype(np.uint32)
    return constants 

def hex_conversion(constants):
    hex_constants = []
    for c in constants:
        hex_constants.append(hex(c)[2:].zfill(8))
    return hex_constants

## test primes
prime_nums = primes(64)
print(prime_nums)
print(len(prime_nums))

## test cubes
cubes = cube_root(prime_nums)
print(cubes)

## test fracs 
frac, nonfrac= np.modf(cubes)
print("fractional:", frac)
print("non fractional:", nonfrac)

## test constants
constants = np.floor(frac * (2**32)).astype(np.uint32)
print("const:", constants)

## test hex constants
hex_constants = hex_conversion(constants)
print("Hex constants", hex_constants)

official_constants = [
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5", "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
    "d807aa98", "12835b01", "243185be", "550c7dc3", "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174", 
    "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc", "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da", 
    "983e5152", "a831c66d", "b00327c8", "bf597fc7", "c6e00bf3", "d5a79147", "06ca6351", "14292967", 
    "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13", "650a7354", "766a0abb", "81c2c92e", "92722c85",
    "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3", "d192e819", "d6990624", "f40e3585", "106aa070",
    "19a4c116", "1e376c08", "2748774c", "34b0bcb5", "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
    "748f82ee", "78a5636f", "84c87814", "8cc70208", "90befffa", "a4506ceb", "bef9a3f7", "c67178f2"
]

def compare_constants(official_constants, hex_constants):
    return np.array_equal(official_constants, hex_constants)

print(len(official_constants))
print(len(hex_constants))
print(official_constants)
print(compare_constants(official_constants, hex_constants))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
64
[1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6.54991162
 6.56541443 6.6418522  6.74599671 6.

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords